# Umgang mit der config.json

Dieses Notebook bündelt Hilfen zum Arbeiten mit der Datei \`C:\\python\\Scripts\\config.json\`.  

Ziele:

- sicher laden (UTF-8), ohne geheime Werte auszugeben
- prüfen, ob \*\*Pflichtfelder\*\* vorhanden und nicht leer sind
- klare Statusmeldungen für den ETL-Betrieb

# Dokumentation: Prüf-Utility für `config.json`

## Zweck

Die Funktion `test_config_file(...)` prüft eine JSON-Konfigurationsdatei auf:

- **Existenz** und **gültiges UTF-8/JSON**
    
- **Pflichtfelder** gemäß einer flexiblen Spezifikation (Spec)
    
- **Wertefreiheit der Ausgabe**: Es werden **keine** sensiblen Werte geloggt – nur Status je Feld.
    

Die Funktion ist **Notebook-freundlich** (kein `sys.exit()`), liefert ein Ergebnis-Dict zur Weiterverarbeitung und kann dennoch als Skript eingesetzt werden.

* * *

## Hauptfunktion

### `test_config_file(config_path, required_spec, *, show_table=True) -> dict`

Prüft die Datei und gibt optional eine kleine Markdown-Tabelle (wertefrei) aus.

**Parameter**

- `config_path` _(str | Path)_: Pfad zur Konfigurationsdatei (z. B. `r"C:\python\Scripts\config.json"`).
    
- `required_spec` _(List\[str\] | Dict\[str, Any\])_: Prüf-Spezifikation (siehe „Spec-Formate“).
    
- `show_table` _(bool, default=True)_: Steuert die Konsolen-/Notebook-Ausgabe einer Status-Tabelle.
    

**Rückgabe (`dict`)**

```
{
  "ok": bool,                    # True, wenn alle Checks bestanden wurden
  "errors": [str, ...],          # Liste textlicher Fehlerbeschreibungen
  "checked": [(path, status), ...]  # z. B. [("sql.server", "✅ ok"), ("sql.password","❌ Leer")]
}

```

**Typische Verwendung**

```
# Spec als Dict (mit Validatoren)
spec = {
  "sharepoint": { "username": "email", "password": "str_nonempty" },
  "sql":        { "username": "str_nonempty", "password": "str_nonempty", "server": "host" }
}
res = test_config_file(r"C:\python\Scripts\config.json", spec)

# Spec als Liste (alle Felder => str_nonempty)
res2 = test_config_file(
    r"C:\python\Scripts\config.json",
    ["sharepoint.username", "sharepoint.password", "sql.username", "sql.password", "sql.server"]
)

```

* * *

## Unterstützte Spec-Formate

### 1) Liste von „dotted paths“

Jeder Pfad wird auf **String, nicht leer (getrimmt)** geprüft (`str_nonempty`).

```
["sharepoint.username", "sharepoint.password", "sql.username", "sql.password", "sql.server"]

```

### 2) Verschachtelte Dict-Struktur mit Validatoren

Leaf-Werte sind Validatoren (Strings). Kurzformen sind möglich.

```
{
  "sharepoint": {
    "username": "email",         # muss E-Mail-ähnlich sein
    "password": "str_nonempty"   # String, getrimmt nicht leer
  },
  "sql": {
    "username": "str_nonempty",
    "password": "str_nonempty",
    "server":   "host"           # grober Host-Check (nicht leer, keine Leerzeichen)
  }
}

```

**Verfügbare Validatoren**

- `"present"`: Key muss existieren (Wert kann leer sein).
    
- `"str_nonempty"`: String, getrimmt nicht leer.
    
- `"email"`: sehr einfache E-Mail-Plausibilitätsprüfung.
    
- `"host"`: einfacher Hostname-Check (nicht leer, keine Leerzeichen).
    
- `"required"` / `True`: Alias für `"str_nonempty"`.
    
- `None`: Alias für `"present"`.
    

* * *

## Hilfsfunktionen (intern)

### `_in_notebook() -> bool`

Erkennt heuristisch, ob der Code im Jupyter-Kernel läuft.  
Wird genutzt, um im Skriptmodus ggf. `sys.exit(...)` zu verwenden – **nicht** in Notebooks.

### `_nested_get(dct: dict, path: Tuple[str, ...]) -> Any`

Sicheres Auslesen verschachtelter Werte entlang eines Pfades, z. B.  
`_nested_get(cfg, ("sql", "server")) → "XYZ"` oder `None`, wenn Key fehlt.  
Wird von den Validatoren verwendet, um Felder robust zu prüfen.

### `_looks_like_email(s: str) -> bool`

Sehr einfache E-Mail-Heuristik (`"@"` vorhanden, Domain hat mindestens einen Punkt).  
Hinweis: Für harte Validierung ggf. robusteren Validator einsetzen.

### `_looks_like_host(s: str) -> bool`

Grobe Host-Plausibilitätsprüfung: **nicht leer**, **keine Leerzeichen**.  
(Kein DNS-Lookup, keine Regex für IP/Hostname – bewusst leichtgewichtig.)

### `_normalize_spec(spec: List[str] | Dict[str, Any]) -> List[PathSpec]`

Normalisiert unterschiedliche Spec-Formate auf eine einheitliche Liste von `(pfad, validator)`\-Tupeln.

- Liste → alles `str_nonempty`
    
- Dict → Leaf-Werte als Validatoren interpretiert; Kurzformen (`True`, `None`, `"required"`) werden aufgelöst.
    

### `_validate_value(val: Any, validator: str) -> Optional[str]`

Validiert einen Wert gegen den angegebenen Validator.

- Rückgabe `None` = ok
    
- Rückgabe `str` = Fehlertext (z. B. `"Fehlend"`, `"Leer"`, `"Falscher Typ (str erwartet)"`, `"Ungültiges Format (...)"`)
    

### `_print_status_table(rows: List[Tuple[str, str]]) -> None`

Gibt eine kleine **wertefreie** Markdown-Tabelle „Pfad | Status“ aus.  
Beispiel:  
`| sql.server | ✅ ok |`, `| sql.password | ❌ Leer |`

* * *

## Verhalten & Eigenschaften

- **Keine Geheimnisse in Logs**: Es werden **keine** Konfigurationswerte ausgegeben, nur Status je Feld.
    
- **Notebook-freundlich**: Kein `sys.exit()`\-Aufruf innerhalb der Prüf-Funktion; Rückgaben sind als Python-Objekt nutzbar.
    
- **Skript-tauglich**: Im `__main__`\-Block kann abhängig von der Umgebung (nicht Notebook) dennoch mit Exit-Codes beendet werden.
    
- **Komplexität**: O(N) in der Anzahl der geprüften Felder; JSON-Parsing ist linear zur Dateigröße.
    

* * *

## Hinweise / Best Practices

- **Secrets** nicht im Klartext loggen; perspektivisch **Secret-Store** (z. B. Azure Key Vault) oder **Umgebungsvariablen** nutzen.
    
- **Validatoren erweitern**: Bei Bedarf im `_validate_value` weitere Fälle ergänzen (z. B. Regex-basierte Prüfungen).
    
- **Striktere Checks** für produktive Pipelines möglich (z. B. TLS-Parametrisierung des SQL-Servers, Domänen-Format für UPNs).

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Konfigurations-Validator für config.json

- test_config_file(path, required_spec, show_table=True) → führt die Prüfung aus (Notebook-freundlich)
- Als Skript nutzbar via __main__ (ohne in Jupyter zu exiten)

required_spec:
  Variante A (Liste von dotted paths):
    ["sharepoint.username", "sharepoint.password", "sql.username", "sql.password", "sql.server"]

  Variante B (verschachtelte Dict mit Validatoren):
    {
      "sharepoint": {
        "username": "email",
        "password": "str_nonempty"
      },
      "sql": {
        "username": "str_nonempty",
        "password": "str_nonempty",
        "server":   "host"
      }
    }
"""

import json
import re
from pathlib import Path
from typing import Any, Dict, Iterable, List, Tuple, Union

# --------------------------------------------
# Helpers
# --------------------------------------------
def _in_notebook() -> bool:
    try:
        from IPython import get_ipython
        ip = get_ipython()
        return ip is not None and "IPKernelApp" in getattr(ip, "config", {})
    except Exception:
        return False

def _nested_get(dct: Dict[str, Any], path: Tuple[str, ...]) -> Any:
    cur: Any = dct
    for key in path:
        if not isinstance(cur, dict) or key not in cur:
            return None
        cur = cur[key]
    return cur

def _looks_like_email(s: str) -> bool:
    return isinstance(s, str) and "@" in s and "." in s.split("@")[-1].strip()

def _looks_like_host(s: str) -> bool:
    return isinstance(s, str) and bool(s.strip()) and " " not in s

# --------------------------------------------
# Spec-Normalisierung
# --------------------------------------------
Validator = str  # "present" | "str_nonempty" | "email" | "host"
PathSpec = Tuple[Tuple[str, ...], Validator]

def _normalize_spec(spec: Union[List[str], Dict[str, Any]]) -> List[PathSpec]:
    """
    Akzeptiert:
      - Liste von dotted paths → Validator "str_nonempty"
      - Verschachtelte Dicts → Leaf = Validator-Name (str),
        True/"required" → "str_nonempty", None → "present"
    """
    out: List[PathSpec] = []

    def push(path: Tuple[str, ...], val: Any):
        # Mappe verschiedene Kurzformen auf Validatoren
        if isinstance(val, str):
            v = val.strip().lower()
            if v in ("present", "str_nonempty", "email", "host"):
                out.append((path, v))
            elif v in ("required",):  # Alias
                out.append((path, "str_nonempty"))
            else:
                # Unbekannter Validator → standardmäßig str_nonempty
                out.append((path, "str_nonempty"))
        elif val is True:
            out.append((path, "str_nonempty"))
        elif val is None:
            out.append((path, "present"))
        else:
            # Fallback
            out.append((path, "str_nonempty"))

    if isinstance(spec, (list, tuple)):
        for dotted in spec:
            if not isinstance(dotted, str):
                continue
            path = tuple(p.strip() for p in dotted.split(".") if p.strip())
            if path:
                out.append((path, "str_nonempty"))
        return out

    if isinstance(spec, dict):
        def walk(node: Dict[str, Any], prefix: Tuple[str, ...]):
            for k, v in node.items():
                if isinstance(v, dict):
                    walk(v, prefix + (k,))
                else:
                    push(prefix + (k,), v)
        walk(spec, tuple())
        return out

    # unbekanntes Format → leer
    return out

# --------------------------------------------
# Validierung
# --------------------------------------------
def _validate_value(val: Any, validator: Validator) -> Union[None, str]:
    """return None wenn ok, sonst Fehlermeldung."""
    if validator == "present":
        if val is None:
            return "Fehlend"
        return None

    if validator == "str_nonempty":
        if val is None:
            return "Fehlend"
        if not isinstance(val, str):
            return "Falscher Typ (str erwartet)"
        if val.strip() == "":
            return "Leer"
        return None

    if validator == "email":
        if val is None:
            return "Fehlend"
        if not isinstance(val, str):
            return "Falscher Typ (str erwartet)"
        if val.strip() == "":
            return "Leer"
        if not _looks_like_email(val):
            return "Ungültiges Format (email erwartet)"
        return None

    if validator == "host":
        if val is None:
            return "Fehlend"
        if not isinstance(val, str):
            return "Falscher Typ (str erwartet)"
        if val.strip() == "":
            return "Leer"
        if not _looks_like_host(val):
            return "Ungültiges Format (host erwartet)"
        return None

    # Unbekannter Validator → als str_nonempty behandeln
    return _validate_value(val, "str_nonempty")

def _print_status_table(rows: List[Tuple[str, str]]) -> None:
    print("\nPflichtfelder (ohne Werte):\n")
    print("| Feld                      | Status  |")
    print("|---------------------------|---------|")
    for dotted, status in rows:
        print(f"| {dotted:<25} | {status:<7} |")

# --------------------------------------------
# Öffentliche Hauptfunktion
# --------------------------------------------
def test_config_file(
    config_path: Union[str, Path],
    required_spec: Union[List[str], Dict[str, Any]],
    *,
    show_table: bool = True,
) -> Dict[str, Any]:
    """
    Prüft die config.json anhand eines flexiblen Specs.

    Returns:
      {
        "ok": bool,
        "errors": [str, ...],
        "checked": [(dotted_path, "ok"/"Fehlend"/... ), ...]
      }
    (Keine Konfigurationswerte werden ausgegeben.)
    """
    p = Path(config_path)

    # 1) Existenz
    if not p.exists():
        msg = f"Datei nicht gefunden: {p}"
        if show_table:
            print(f"❌ {msg}")
        return {"ok": False, "errors": [msg], "checked": []}

    # 2) Laden
    try:
        raw = p.read_text(encoding="utf-8")
        cfg = json.loads(raw)
    except UnicodeDecodeError as e:
        msg = f"Keine UTF-8-Kodierung: {e}"
        if show_table:
            print(f"❌ {msg}")
        return {"ok": False, "errors": [msg], "checked": []}
    except json.JSONDecodeError as e:
        msg = f"Ungültiges JSON: {e}"
        if show_table:
            print(f"❌ {msg}")
        return {"ok": False, "errors": [msg], "checked": []}

    # 3) Spec normalisieren
    checks = _normalize_spec(required_spec)

    # 4) Prüfen
    checked_rows: List[Tuple[str, str]] = []
    errors: List[str] = []
    for path, validator in checks:
        dotted = ".".join(path)
        val = _nested_get(cfg, path)
        err = _validate_value(val, validator)
        if err is None:
            status = "✅ ok"
        else:
            status = f"❌ {err}"
            errors.append(f"{dotted}: {err}")
        checked_rows.append((dotted, status))

    # 5) Ausgabe (ohne Werte)
    if show_table:
        print(f"Konfigurationsdatei gefunden: {p}")
        _print_status_table(checked_rows)
        if errors:
            print("\nValidierungsfehler:")
            for e in errors:
                print(" -", e)
        else:
            print("\n✅ Validierung OK. (Keine Konfigurationswerte ausgegeben.)")

    return {"ok": len(errors) == 0, "errors": errors, "checked": checked_rows}

# Beispiel 1: Testen auf eine bestimmte Spezifikation

In [5]:
# --------------------------------------------
# Beispiel-Default für direkten Skript-Run
# --------------------------------------------
path = Path(r"C:\python\Scripts\config.json")
spec = {
    "sharepoint": {
        "username": "email",
        "password": "str_nonempty",
    },
    "sql": {
        "username": "str_nonempty",
        "password": "str_nonempty",
        "server":   "host",
    }
}

if __name__ == "__main__":
    # Skript-Verhalten: keine harten Exits in Jupyter
    result = test_config_file(DEFAULT_PATH, DEFAULT_SPEC, show_table=True)
    if not _in_notebook():
        import sys
        sys.exit(0 if result["ok"] else 2)

Konfigurationsdatei gefunden: C:\python\Scripts\config.json

Pflichtfelder (ohne Werte):

| Feld                      | Status  |
|---------------------------|---------|
| sharepoint.username       | ✅ ok    |
| sharepoint.password       | ✅ ok    |
| sql.username              | ✅ ok    |
| sql.password              | ✅ ok    |
| sql.server                | ✅ ok    |

✅ Validierung OK. (Keine Konfigurationswerte ausgegeben.)
